In [76]:
from functions import *

In [77]:
params = read_parameters()
file_idx_uppaal = params['file_idx_uppaal']
plot_scatterplot_matrix = params['plot_scatterplot_matrix']

subfolder: UF/
file_idx_uppaal: 0
file_idx: 0
tmp_idx: 0
initial_feed_volume_L: 100
reset_columns_when_OFF: True
drop_initial_final_off_rows: True
drop_off_rows: True
drop_outliers: True
plot_scatterplot_matrix: False
use_default_arima_params: True
default_arima_params: [1, 1, 0]
include_arima_simulations_in_analysis: True


In [79]:
in_folder = PATH_UPPAAL_DATA_RAW
out_folder_1 = PATH_UPPAAL_DATA_RAW_REPAIRED
out_folder_2 = PATH_UPPAAL_DATA_EXT
in_files = [ (in_folder, file) for file in os.listdir(in_folder) if re.match(".*\.csv", file) ]

in_file_name = in_files[file_idx_uppaal][1]
in_file_path = in_folder + in_file_name
out_file_path_1 = out_folder_1 + in_file_name
out_file_path_2 = out_folder_2 + in_file_name

print("file list:")
for i in range(len(in_files)) :
    print(i, in_files[i][0] + in_files[i][1])
print(f"\ninput file:\n{in_file_path}")

file list:
0 /workspaces/data_analysis/data/from_uppaal/0_raw/k_n=5.5e-6,n=0,flux_min=res,res_0=4e12.csv
1 /workspaces/data_analysis/data/from_uppaal/0_raw/k_n=5e-6,n=0,flux_min=res,res_0=3.3e12.csv
2 /workspaces/data_analysis/data/from_uppaal/0_raw/k_n=5e-6,n=0,flux_min=res,res_0=4e12.csv
3 /workspaces/data_analysis/data/from_uppaal/0_raw/k_n=5e-6,n=0,flux_min=res.csv
4 /workspaces/data_analysis/data/from_uppaal/0_raw/k_n=5e-6,n=0.csv
5 /workspaces/data_analysis/data/from_uppaal/0_raw/k_n=6e-6,n=0,flux_min=res,res_0=3.8e12.csv

input file:
/workspaces/data_analysis/data/from_uppaal/0_raw/k_n=6e-6,n=0,flux_min=res,res_0=3.8e12.csv


In [80]:
in_file = open(in_file_path, "r")
out_file_1 = open(out_file_path_1, "w")
orig_stdout = sys.stdout
sys.stdout = out_file_1
for line in in_file:
    if(re.match('^\#.*[^,]$', line)) :
        print(f'{line[:-1]},')
    else :
        print(line, end='')
sys.stdout = orig_stdout
in_file.close()
out_file_1.close()

In [81]:
COLUMNS_TO_KEEP = [
    'time [m]', 'prs_TMP', 'flux', 'flux_min', 'res_tot', 'flux_slope', 
    'flow_prmt', 'flow_rtnt', 'flow_feed', 
    'energy_CP', 'energy_PP', 
    'vol_feed', 'vol_prmt',
    'temperature', 'FIXED_TEMPERATURE', 'viscosity', 'prs_rtnt'
]
# 'time_since_update', 
# 'conc_prmt', 'energy_per_m3', , 'temperature_sqrt', 
# 'time_hrs', , 'AREA_TOT', 'conc_feed_0', 'vol_feed_0', 'weight_feed_0'
# 'prs_feed_1', 'prs_feed_2', 'prs_rtnt','prs_TMP_delta', 'conc_feed', 'time_min', 

In [82]:
df_all = pd.read_csv(out_file_path_1, skiprows=1, names=['time [m]', 'value'])
df_all['is_header'] = df_all['time [m]'].apply(lambda t : t[0] == '#')
init_rows = np.array(df_all[df_all['is_header'] == True].index)
init_rows = np.append(init_rows, len(df_all))
out_df = None
for i in range(len(init_rows)-1) :
    df = pd.DataFrame()
    cur_idx = init_rows[i]
    nxt_idx = init_rows[i+1]
    size = (nxt_idx-cur_idx-1)
    col_name = df_all.loc[cur_idx, 'time [m]'][2:-3]
    keep_col = col_name in COLUMNS_TO_KEEP
    keep_col_msg = 'kept' if keep_col else 'dropped'
    print(f'column: {col_name}, size: {size} -> {keep_col_msg}')
    if  keep_col :
        df['time [m]'] = (df_all['time [m]'][cur_idx+1:nxt_idx]).astype(float)
        df[col_name] = (df_all['value'][cur_idx+1:nxt_idx]).astype(float)
        df = df.drop_duplicates()
        df['time equal as above'] = df['time [m]'].diff() < 0.01
        duplicate_rows = df[(df['time equal as above'] == True) & (df[col_name].diff() < 1e-8)].index
        df = df.drop(duplicate_rows, axis=0)
        df['time [m]'] = df.apply(lambda row : row['time [m]'] if row['time equal as above'] == False else row['time [m]'] + 0.01, axis=1)
        df['time [m]'] = (100 * df['time [m]']).astype(int)
        df = df.drop(labels=['time equal as above'], axis=1)
        df = df.drop_duplicates()
        if out_df is None :
            out_df = df
        else :
            out_df = pd.merge(out_df, df, how='outer', on='time [m]')
out_df['time [m]'] = out_df['time [m]'].astype(float) / 100
out_df['time [m]'] = out_df['time [m]'].apply(lambda t : t+0.01 if round(100*(t+0.01)) % 10 == 0 else t)
out_df = out_df.sort_values(by=['time [m]']).reset_index(drop=True)
out_df = out_df[1:].reset_index(drop=True)
df = out_df

column: conc_feed, size: 1498 -> dropped
column: conc_prmt, size: 384 -> dropped
column: flow_feed, size: 1527 -> kept
column: flow_prmt, size: 1527 -> kept
column: flux, size: 1527 -> kept
column: flux_slope, size: 384 -> kept
column: energy_CP, size: 1412 -> kept


column: energy_PP, size: 1229 -> kept
column: energy_per_m3, size: 384 -> dropped
column: res_tot, size: 1543 -> kept
column: temperature, size: 384 -> kept
column: temperature_sqrt, size: 384 -> dropped
column: viscosity, size: 384 -> kept
column: vol_feed, size: 1527 -> kept
column: vol_prmt, size: 1527 -> kept
column: time_min, size: 384 -> dropped
column: time_hrs, size: 384 -> dropped
column: time_since_update, size: 384 -> dropped
column: FIXED_TEMPERATURE, size: 384 -> kept
column: AREA_TOT, size: 384 -> dropped
column: conc_feed_0, size: 384 -> dropped
column: flow_rtnt, size: 384 -> kept
column: flux_min, size: 384 -> kept
column: prs_rtnt, size: 384 -> kept
column: prs_feed_1, size: 384 -> dropped
column: prs_feed_2, size: 384 -> dropped
column: prs_TMP, size: 384 -> kept
column: prs_TMP_delta, size: 384 -> dropped
column: vol_feed_0, size: 384 -> dropped
column: weight_feed_0, size: 384 -> dropped


In [83]:
TIME_IDX  = ROW_IDX_AS_TIME
x_ax_lbl = 'time [m]' if TIME_IDX == ROW_IDX_AS_TIME else x_axis[TIME_IDX]

In [84]:
CONST_TEMPERATURE = (df['FIXED_TEMPERATURE'].dropna().iloc[0] == 1.0)
TEMPERATURE = int(df['temperature'].dropna().iloc[0])
FLUX_COL = f'flux at {TEMPERATURE}° [L/m^2h]' if CONST_TEMPERATURE else 'flux [L/m^2h]'
FLUX_USED = FLUX_COL[:-9]
FLUX_MIN_COL = f'{FLUX_USED} min [L/m^2h]'
print(FLUX_COL)
print(FLUX_USED)

flux at 20° [L/m^2h]
flux at 20°


In [85]:
df = df.rename({
    'flow_feed' :       'flow feed [L/h]',
    'flow_prmt' :       'flow permeate [L/h]',
    'flux' :            FLUX_COL,
    'flux_slope' :      'k(n=2)',
    'energy_CP' :       'energy CP [kWh]',
    'energy_PP' :       'energy PP [kWh]',
    'res_tot' :         'res tot [1/m]',
    'temperature' :     'temperature [°C]',
    'viscosity' :       'viscosity permeate [Pa s]',
    'vol_feed' :        'vol feed [L]',
    'vol_prmt' :        'vol permeate [L]',
    'FIXED_TEMPERATURE' :  'is temperature fixed',
    'flow_rtnt' :        'flow retentate [L/h]',
    'flux_min' :        f'{FLUX_USED} min [L/m^2h]',
    'prs_TMP' :        'TMP [kPa]',
    'prs_rtnt' :        'prs retentate [kPa]',
}, axis=1)

In [86]:
DISCRETE_COLUMNS = [
    'is temperature fixed', 'flow retentate [L/h]', f'{FLUX_USED} min [L/m^2h]', 'TMP [kPa]', 'prs retentate [kPa]',
]
if CONST_TEMPERATURE :
    DISCRETE_COLUMNS.append('temperature [°C]')
    DISCRETE_COLUMNS.append('viscosity permeate [Pa s]')
for c in DISCRETE_COLUMNS:
    df[c] = df[c].ffill()

In [87]:
cur_TMP = 330
df[f'flux at 20° TMP={cur_TMP} [L/m^2h]'] = df['flux at 20° [L/m^2h]'] * (cur_TMP / df['TMP [kPa]'])

In [88]:
BUCKET_SIZE = 50
df['TMP class'] = df['TMP [kPa]'].astype(int).apply(lambda x : x // BUCKET_SIZE)
df['TMP interval'] = df['TMP class'].apply(lambda x : f'{BUCKET_SIZE*x}-{BUCKET_SIZE*(x+1)-1}')
df['TMP interval'] = df['TMP class'].apply(lambda x : BUCKET_SIZE*x)

In [89]:
df.to_csv(out_file_path_2, index=False)

In [90]:
if plot_scatterplot_matrix :
    hue = None
    corr_cols = [
        'time [m]', 'flux at 20° [L/m^2h]', 'flux at 20° TMP=330 [L/m^2h]',
        'energy CP [kWh]', 'energy PP [kWh]', 'res tot [1/m]', 
        'flow retentate [L/h]', 'flux at 20° min [L/m^2h]', 'TMP [kPa]',
        'TMP interval'
    ]
    hue='TMP interval' #'TMP [kPa]'
    #if len(conc_lines) > 1 :
    #    # at least two concentrations
    #    hue = 'initial feed concentration [g/L]'
    #    corr_cols.append('initial feed concentration [g/L]')
    sn.pairplot(df[corr_cols], hue=hue)